In [8]:
# Cell 1 — Get Top 10 Steam games by all-time owners (SteamSpy)

import requests

# 1) Pull the “top100forever” snapshot from SteamSpy
url  = "https://steamspy.com/api.php"
resp = requests.get(url, params={"request":"top100forever"}, timeout=10)
resp.raise_for_status()
data = resp.json()  # dict of { "570": {...}, "730": {...}, … }

# 2) Sort by the upper bound of the "owners" field ("min..max")
def owners_upper(o_str: str) -> int:
    try:
        return int(o_str.split("..")[-1].replace(",", ""))
    except:
        return 0

games = list(data.values())
games_sorted = sorted(
    games,
    key=lambda g: owners_upper(g.get("owners", "")),
    reverse=True
)

# 3) Take the top 10 entries
top10 = games_sorted[:10]

# 4) Print & collect their AppIDs *and* names
print("Top 10 Steam Games by all-time owners:")
TOP10_GAMES = []   # list of (appid, name)
for g in top10:
    aid    = g["appid"]
    name   = g.get("name", "<unknown>")
    owners = g.get("owners", "<n/a>")
    print(f" • {aid}: {name}  (owners ≈ {owners})")
    TOP10_GAMES.append((aid, name))

print("\n▶ Using these games for backfill:", TOP10_GAMES)

Top 10 Steam Games by all-time owners:
 • 730: Counter-Strike: Global Offensive  (owners ≈ 100,000,000 .. 200,000,000)
 • 1172470: Apex Legends  (owners ≈ 100,000,000 .. 200,000,000)
 • 578080: PUBG: BATTLEGROUNDS  (owners ≈ 100,000,000 .. 200,000,000)
 • 1623730: Palworld  (owners ≈ 50,000,000 .. 100,000,000)
 • 440: Team Fortress 2  (owners ≈ 50,000,000 .. 100,000,000)
 • 1938090: Call of Duty: Modern Warfare II  (owners ≈ 50,000,000 .. 100,000,000)
 • 1063730: New World: Aeternum  (owners ≈ 50,000,000 .. 100,000,000)
 • 2358720: Black Myth: Wukong  (owners ≈ 50,000,000 .. 100,000,000)
 • 271590: Grand Theft Auto V Legacy  (owners ≈ 50,000,000 .. 100,000,000)
 • 550: Left 4 Dead 2  (owners ≈ 50,000,000 .. 100,000,000)

▶ Using these games for backfill: [(730, 'Counter-Strike: Global Offensive'), (1172470, 'Apex Legends'), (578080, 'PUBG: BATTLEGROUNDS'), (1623730, 'Palworld'), (440, 'Team Fortress 2'), (1938090, 'Call of Duty: Modern Warfare II'), (1063730, 'New World: Aeternum'), (2

In [10]:
# Cell 2 — Fetch most‐recent 150 pages, but inject `app_id` & `game_name` into each JSON record
import requests, time, json, re

def slugify(name: str) -> str:
    s = name.lower()
    s = re.sub(r'[^a-z0-9]+', '_', s)
    return re.sub(r'_{2,}', '_', s).strip('_') or "unknown"

def fetch_recent_reviews(
    appid: int,
    game_name: str,
    per_page: int  = 100,
    max_pages: int = 30,
    pause: float   = 0.2,
    outfile: str   = None
) -> str:
    slug    = slugify(game_name)
    outfile = outfile or f"steam_{appid}_{slug}_reviews_recent.jsonl"

    base_url = f"https://store.steampowered.com/appreviews/{appid}"
    params   = {
        "json":          "1",
        "language":      "english",
        "filter":        "recent",
        "num_per_page":  str(per_page),
    }
    cursor  = None
    page    = 0
    written = 0

    with open(outfile, "w", encoding="utf-8") as f:
        while page < max_pages:
            page += 1
            if cursor:
                params["cursor"] = cursor

            resp    = requests.get(base_url, params=params,
                                   headers={"User-Agent":"Mozilla/5.0"})
            resp.raise_for_status()
            data    = resp.json()
            reviews = data.get("reviews", [])
            if not reviews:
                print(f"→ no reviews on page {page}, stopping.")
                break

            print(f"→ page {page}/{max_pages}, got {len(reviews)} reviews.")

            for r in reviews:
                # **INJECT** the app_id & game_name into each record:
                r["app_id"]    = appid
                r["game_name"] = game_name
                f.write(json.dumps(r, ensure_ascii=False) + "\n")
                written += 1

            cursor = data.get("cursor", "")
            if not cursor:
                print("→ no next cursor; done.")
                break

            time.sleep(pause)

    print(f"\n✅ Done: wrote {written:,} reviews to {outfile}")
    return outfile

# — Run this for each game in your TOP10_GAMES list:
for aid, name in TOP10_GAMES:
    fetch_recent_reviews(
        appid     = aid,
        game_name = name,
        per_page  = 100,
        max_pages = 200,
        pause     = 0.2
    )

→ page 1/200, got 100 reviews.
→ page 2/200, got 100 reviews.
→ page 3/200, got 100 reviews.
→ page 4/200, got 100 reviews.
→ page 5/200, got 100 reviews.
→ page 6/200, got 100 reviews.
→ page 7/200, got 100 reviews.
→ page 8/200, got 100 reviews.
→ page 9/200, got 100 reviews.
→ page 10/200, got 100 reviews.
→ page 11/200, got 100 reviews.
→ page 12/200, got 100 reviews.
→ page 13/200, got 100 reviews.
→ page 14/200, got 100 reviews.
→ page 15/200, got 100 reviews.
→ page 16/200, got 100 reviews.
→ page 17/200, got 100 reviews.
→ page 18/200, got 100 reviews.
→ page 19/200, got 100 reviews.
→ page 20/200, got 100 reviews.
→ page 21/200, got 100 reviews.
→ page 22/200, got 100 reviews.
→ page 23/200, got 100 reviews.
→ page 24/200, got 100 reviews.
→ page 25/200, got 100 reviews.
→ page 26/200, got 100 reviews.
→ page 27/200, got 100 reviews.
→ page 28/200, got 100 reviews.
→ page 29/200, got 100 reviews.
→ page 30/200, got 100 reviews.
→ page 31/200, got 100 reviews.
→ page 32/200, go

In [12]:
# Cell 3 — Upload recent‐reviews JSONL files to GCS
from google.cloud import storage
import glob, os

# 1) configure
BUCKET_NAME = "steam-reviews-bucket-0"          # ← your real bucket name only
PREFIX      = "top-10-reviews-steamcommunity/backfill/recent_reviews/"

# 2) init
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

# 3) upload
for path in glob.glob("steam_*_reviews_recent.jsonl"):
    blob_path = os.path.join(PREFIX, os.path.basename(path))
    blob = bucket.blob(blob_path)
    print(f"Uploading {path} → gs://{BUCKET_NAME}/{blob_path}")
    blob.upload_from_filename(path)

print("✅ All files uploaded.")

Uploading steam_2358720_black_myth_wukong_reviews_recent.jsonl → gs://steam-reviews-bucket-0/top-10-reviews-steamcommunity/backfill/recent_reviews/steam_2358720_black_myth_wukong_reviews_recent.jsonl
Uploading steam_1172470_apex_legends_reviews_recent.jsonl → gs://steam-reviews-bucket-0/top-10-reviews-steamcommunity/backfill/recent_reviews/steam_1172470_apex_legends_reviews_recent.jsonl
Uploading steam_578080_pubg_battlegrounds_reviews_recent.jsonl → gs://steam-reviews-bucket-0/top-10-reviews-steamcommunity/backfill/recent_reviews/steam_578080_pubg_battlegrounds_reviews_recent.jsonl
Uploading steam_1623730_palworld_reviews_recent.jsonl → gs://steam-reviews-bucket-0/top-10-reviews-steamcommunity/backfill/recent_reviews/steam_1623730_palworld_reviews_recent.jsonl
Uploading steam_730_counter_strike_global_offensive_reviews_recent.jsonl → gs://steam-reviews-bucket-0/top-10-reviews-steamcommunity/backfill/recent_reviews/steam_730_counter_strike_global_offensive_reviews_recent.jsonl
Uploadin

In [13]:
# Cell 4 — Ensure dataset/table & then load your JSONL into BigQuery (ignoring extra fields)
import json
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

# 1) Initialize client & IDs
client     = bigquery.Client(project="sentiment-analysis-steam")
dataset_id = "sentiment-analysis-steam.steam_reviews"
# **FIXED**: use underscores, match the actual table name you created
table_id   = f"{dataset_id}.top10-owned-steamcommunity"

# 2) Create the dataset if it doesn’t exist
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)

# 3) Define your schema exactly as before
schema = [
    bigquery.SchemaField("recommendationid",    "STRING"),
    bigquery.SchemaField("author",              "RECORD", mode="NULLABLE", fields=[
        bigquery.SchemaField("steamid",                  "STRING"),
        bigquery.SchemaField("num_games_owned",          "INT64"),
        bigquery.SchemaField("num_reviews",              "INT64"),
        bigquery.SchemaField("playtime_forever",         "INT64"),
        bigquery.SchemaField("playtime_last_two_weeks",  "INT64"),
        bigquery.SchemaField("playtime_at_review",       "INT64"),
        bigquery.SchemaField("last_played",              "INT64"),
    ]),
    bigquery.SchemaField("language",                  "STRING"),
    bigquery.SchemaField("review",                    "STRING"),
    bigquery.SchemaField("timestamp_created",         "INT64"),
    bigquery.SchemaField("timestamp_updated",         "INT64"),
    bigquery.SchemaField("voted_up",                  "BOOL"),
    bigquery.SchemaField("votes_up",                  "INT64"),
    bigquery.SchemaField("votes_funny",               "INT64"),
    bigquery.SchemaField("weighted_vote_score",       "FLOAT"),
    bigquery.SchemaField("comment_count",             "INT64"),
    bigquery.SchemaField("steam_purchase",            "BOOL"),
    bigquery.SchemaField("received_for_free",         "BOOL"),
    bigquery.SchemaField("written_during_early_access","BOOL"),
    bigquery.SchemaField("primarily_steam_deck",      "BOOL"),
    bigquery.SchemaField("app_id",                    "INT64"),
    bigquery.SchemaField("game_name",                 "STRING"),
]

# 4) Create the table if it doesn’t exist
try:
    client.get_table(table_id)
    print(f"✅ Table {table_id} already exists.")
except NotFound:
    table = bigquery.Table(table_id, schema=schema)
    client.create_table(table)
    print(f"✅ Created table {table_id}.")

# 5) Configure the load to ignore any unknown JSON fields
job_config = bigquery.LoadJobConfig(
    source_format         = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition     = bigquery.WriteDisposition.WRITE_TRUNCATE,
    schema                = schema,
    ignore_unknown_values = True,    # drop extra keys
)

# 6) URI pointing at your uploaded JSONL files
uri = "gs://steam-reviews-bucket-0/top-10-reviews-steamcommunity/backfill/recent_reviews/*.jsonl"

# 7) Kick off the load job
load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)
print("📥 Starting BigQuery load (ignoring unknown values)…")
load_job.result()  # wait for it to finish

# 8) Confirm how many rows landed
table = client.get_table(table_id)
print(f"✅ Loaded {table.num_rows:,} rows into {table_id}.")
# and if you want to see any load errors:
if load_job.errors:
    print("⚠️  Load errors:", load_job.errors)

✅ Table sentiment-analysis-steam.steam_reviews.top10-owned-steamcommunity already exists.
📥 Starting BigQuery load (ignoring unknown values)…
✅ Loaded 181,014 rows into sentiment-analysis-steam.steam_reviews.top10-owned-steamcommunity.
